In [78]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


df = pd.read_csv('../input/depression-anxiety-stress-scales/DASS_data_21.02.19/data.csv', sep=r'\t', engine='python')

df = df[df.familysize <= 25]
df = df[(df.VCL9 == 0) | (df.VCL6 == 0) | (df.VCL12 == 0)]
df = df[df.age <= 100]
df = df.drop('major', 1)

df["Total_time"] = df.apply(lambda row: (row.Q1E + row.Q2E + row.Q3E + row.Q4E + row.Q5E + 
                                         row.Q6E + row.Q7E + row.Q8E +row.Q9E + 
                                         row.Q10E + row.Q11E + row.Q12E + row.Q13E +                                         
                                         row.Q14E + row.Q15E + row.Q16E + row.Q17E + 
                                         row.Q18E + row.Q19E + row.Q20E + row.Q21E +
                                         row.Q22E + row.Q23E + row.Q24E + row.Q25E +
                                         row.Q26E + row.Q27E + row.Q28E + row.Q29E + 
                                         row.Q30E + row.Q31E + row.Q32E + row.Q33E +
                                         row.Q34E + row.Q35E + row.Q36E + row.Q37E +
                                         row.Q38E + row.Q39E + row.Q40E + row.Q41E +
                                         row.Q42E)/1000, axis=1)

df = df[(df.Total_time <= 3600) & (df.Total_time >= 100) ]

l = ["Q1A","Q2A","Q3A","Q4A","Q5A","Q6A","Q7A","Q8A","Q9A","Q10A",
     "Q11A","Q12A","Q13A","Q14A","Q15A","Q16A","Q17A","Q18A","Q19A",
     "Q20A" ,"Q21A","Q22A","Q23A","Q24A","Q25A" ,"Q26A","Q27A","Q28A",
     "Q29A","Q30A","Q31A","Q32A","Q33A","Q34A","Q35A","Q36A","Q37A","Q38A",
     "Q39A","Q40A","Q41A","Q42A"]



for i in l:
    
    df.loc[df[i] == 1, i] = 0
    df.loc[df[i] == 2, i] = 2
    df.loc[df[i] == 3, i] = 4
    df.loc[df[i] == 4, i] = 6

df["Depression_score"] = df.apply(lambda row: row.Q3A + row.Q5A +  row.Q10A + row.Q13A + row.Q16A + row.Q17A + row.Q21A +row.Q24A +   row.Q26A +  row.Q31A + row.Q34A + row.Q37A + row.Q38A + row.Q42A , axis=1)
df["Anxiety_score"] = df.apply(lambda row: row.Q2A + row.Q4A +  row.Q7A + row.Q9A + row.Q15A + row.Q19A + row.Q20A + row.Q23A +row.Q25A + row.Q28A +  row.Q30A + row.Q36A + row.Q40A + row.Q41A , axis=1)
df["Stress_score"] = df.apply(lambda row: row.Q1A + row.Q6A +  row.Q8A + row.Q11A + row.Q12A + row.Q14A + row.Q18A + row.Q22A +row.Q27A + row.Q29A +  row.Q32A + row.Q33A + row.Q35A + row.Q39A , axis=1)
df.drop(df.columns[:126], axis = 1, inplace = True)
df.drop(df.columns[15:31],axis = 1, inplace = True)

df["gender"].replace({0: 2}, inplace=True)
df["gender"].replace({1: 'Male', 2: 'Female', 3 : 'Other'}, inplace=True)
df["urban"].replace({0: 3}, inplace=True)
df["urban"].replace({1: 'Rural', 2: 'Suburban', 3 : 'Urban'}, inplace=True)
df["orientation"].replace({0: 1}, inplace=True)
df["orientation"].replace({1: 'Heterosexual', 2: 'Bisexual',3: 'Homosexual',4: 'Asexual', 5 : 'Other'}, inplace=True)
df["married"].replace({0: 1}, inplace=True)
df["married"].replace({1: 'Single', 2: 'Married', 3 : 'Divorced'}, inplace=True)
df["religion"].replace({0: 10}, inplace=True)
df["race"].replace({10: 'Asian', 20: 'Arab', 30 : 'Black', 40 : 'Indigenous Australian', 50 : 'Native American', 60 : 'White', 70 : 'Other'}, inplace=True)
df["religion"].replace({1: 'Agnostic', 2: 'Atheist', 3 : 'Bhuddist',  4 : 'Christian', 5 : 'Christian', 6 : 'Christian', 7 : 'Christian', 8 : 'Hindu', 9 : 'Jewish', 10 : 'Muslim',11 : 'Sikh', 12: 'Other' }, inplace=True)
df = df.drop(['introelapse', 'testelapse','surveyelapse','Total_time','voted','country','engnat'], axis=1)

print(df)

TIPI4 Anxious, easily upset.
TIPI5 Open to new experiences, complex.
TIPI6 Reserved, quiet.
TIPI7 Sympathetic, warm.
TIPI8 Disorganized, careless.
TIPI9 Calm, emotionally stable.
TIPI10 Conventional, uncreative.

In [79]:
df.dtypes
df_classification = df

In [80]:
df_classification.rename(columns={'TIPI1': 'Extroverted_Enthusiastic', 'TIPI2': 'Critical_Quarrelsome','TIPI3': 'Dependable_Self-disciplined', 'TIPI4': 'Anxious_Easily_upset','TIPI5': 'Open to new experiences_Complex', 'TIPI6': 'Reserved_Quiet','TIPI7': 'Sympathetic_Warm', 'TIPI8': 'Disorganized_Careless','TIPI9': 'Calm_Emotionally_stable', 'TIPI10': 'Conventional_Uncreative'}, inplace=True)

In [81]:
df_classification

In [82]:
df = pd.get_dummies(df,prefix=['gender'], columns = ['gender'])
df = pd.get_dummies(df,prefix=['religion'], columns = ['religion'])
df = pd.get_dummies(df,prefix=['orientation'], columns = ['orientation'])
df = pd.get_dummies(df,prefix=['married'], columns = ['married'])
df = pd.get_dummies(df,prefix=['race'], columns = ['race'])
df= pd.get_dummies(df, prefix=['urban'], columns = ['urban'])
df.dtypes



In [83]:
df_regression = df
print(df_regression)

In [84]:

filters = [
   (df_classification.Depression_score <= 18),
   (df_classification.Depression_score > 18) & (df_classification.Depression_score <= 26),
    (df_classification.Depression_score > 26) & (df_classification.Depression_score <= 40),
    (df_classification.Depression_score > 40) & (df_classification.Depression_score <= 54),
   (df_classification.Depression_score > 55)
]
values = ["Normal", "Mild", "Moderate","Severe","Extremely Severe"]

df_classification["Depression_cat"] = np.select(filters, values)

filters = [
   (df_classification.Anxiety_score <= 18),
   (df_classification.Anxiety_score > 18) & (df_classification.Anxiety_score <= 26),
    (df_classification.Anxiety_score > 26) & (df_classification.Anxiety_score <= 40),
    (df_classification.Anxiety_score > 40) & (df_classification.Anxiety_score <= 54),
   (df_classification.Anxiety_score > 55)
]
values = ["Normal", "Mild", "Moderate","Severe","Extremely Severe"]

df_classification["Anxiety_cat"] = np.select(filters, values)

filters = [
   (df_classification.Stress_score <= 18),
   (df_classification.Stress_score > 18) & (df_classification.Stress_score <= 26),
    (df_classification.Stress_score > 26) & (df_classification.Stress_score <= 40),
    (df_classification.Stress_score > 40) & (df_classification.Stress_score <= 54),
   (df_classification.Stress_score > 55)
]
values = ["Normal", "Mild", "Moderate","Severe","Extremely Severe"]

df_classification["Stress_cat"] = np.select(filters, values)

print(df_classification)

In [85]:
df_depression = df_classification.drop(['Stress_cat', 'Anxiety_cat','Depression_score','Anxiety_score','Stress_score'], axis=1)
df_anxiety = df_classification.drop(['Depression_cat','Stress_cat','Depression_score','Anxiety_score','Stress_score'], axis = 1)
df_stress = df_classification.drop(['Anxiety_cat','Depression_cat','Depression_score','Anxiety_score','Stress_score'], axis = 1)
print(df_stress)

In [86]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style("whitegrid")

#gender_depression
grouped = df_classification.loc[:,['gender', 'Depression_score']] \
    .groupby(['gender']) \
    .median() \
    .sort_values(by='Depression_score')
ax1_d = sns.boxplot(x = df_classification["gender"], y = df_classification["Depression_score"],order=grouped.index,data = df_classification)
ax1_d.set_xlabel("Gender", fontsize = 20)
ax1_d.set_ylabel("Depression Score", fontsize = 20)
ax1_d.set_title("Non-Binary people tend to have higher depression score",fontsize= 30)

ax1_d.figure.savefig("gender_d")

#gender_anxiety
grouped = df_classification.loc[:,['gender', 'Anxiety_score']] \
    .groupby(['gender']) \
    .median() \
    .sort_values(by='Anxiety_score')
ax1_a = sns.boxplot(x = df_classification["gender"], y = df_classification["Anxiety_score"],order = grouped.index, data = df_classification)
ax1_a.set_xlabel("Gender", fontsize = 20)
ax1_a.set_ylabel("Anxiety Score", fontsize = 20)
ax1_a.set_title("Non-Binary people tend to have higher anxiety score",fontsize= 30)
ax1_a.figure.savefig("gender_a")

#gender_stress
grouped = df_classification.loc[:,['gender', 'Stress_score']] \
    .groupby(['gender']) \
    .median() \
    .sort_values(by='Stress_score')

ax1_s = sns.boxplot(x = df_classification["gender"], y = df_classification["Stress_score"], order= grouped.index, data = df_classification)
ax1_s.set_xlabel("Gender", fontsize = 20)
ax1_s.set_ylabel("Stress Score", fontsize = 20)
ax1_s.set_title("Non-Binary people tend to have higher stress score",fontsize= 30)
ax1_s.figure.savefig("gender_s")

In [87]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style("whitegrid")

#religion_depression
grouped = df_classification.loc[:,['religion', 'Depression_score']] \
    .groupby(['religion']) \
    .median() \
    .sort_values(by='Depression_score')
ax2_d = sns.boxplot(x = df_classification["religion"], y = df_classification["Depression_score"],order= grouped.index,data = df_classification)
ax2_d.set_xlabel("Religion", fontsize = 20)
ax2_d.set_ylabel("Depression Score", fontsize = 20)
ax2_d.set_title("Atheists tend to have higher depression score",fontsize= 30)
ax2_d.figure.savefig("religion_d")

#religion_anxiety


grouped = df_classification.loc[:,['religion', 'Anxiety_score']] \
    .groupby(['religion']) \
    .median() \
    .sort_values(by='Anxiety_score')

ax2_a = sns.boxplot(x = df_classification["religion"], y = df_classification["Anxiety_score"],order = grouped.index,data = df_classification)
ax2_a.set_xlabel("Religion", fontsize = 20)
ax2_a.set_ylabel("Anxiety Score", fontsize = 20)
ax2_a.set_title("Sikh and people from other religion tend to have higher anxiety score",fontsize= 30)
ax2_a.figure.savefig("religion_a")

#religion_stress



grouped = df_classification.loc[:,['religion', 'Stress_score']] \
    .groupby(['religion']) \
    .median() \
    .sort_values(by='Stress_score')

ax2_s = sns.boxplot(x = df_classification["religion"], y = df_classification["Stress_score"],order = grouped.index,data = df_classification)
ax2_s.set_xlabel("Religion", fontsize = 20)
ax2_s.set_ylabel("Stress Score", fontsize = 20)
ax2_s.set_title("Sikhs tend to have higher stress score",fontsize= 30)
ax2_s.figure.savefig("religion_s")

In [88]:


ax3_d = sns.lineplot(x = df_classification["age"], y = df_classification["Depression_score"],ci= None,data = df_classification)
ax3_d.set_xlabel("Age", fontsize = 20)
ax3_d.set_ylabel("Depression Score", fontsize = 20)
ax3_d.set_title("As age increases, depression score tends to decrease",fontsize= 30)

In [89]:
ax3_a = sns.lineplot(x = df_classification["age"], y = df_classification["Anxiety_score"],ci= None,data = df_classification)
ax3_a.set_xlabel("Age", fontsize = 20)
ax3_a.set_ylabel("Anxiety Score", fontsize = 20)
ax3_a.set_title("As age increases, Anxiety score tends to decrease",fontsize= 30)

In [90]:
ax3_s = sns.lineplot(x = df_classification["age"], y = df_classification["Stress_score"],ci= None,data = df_classification)
ax3_s.set_xlabel("Age", fontsize = 20)
ax3_s.set_ylabel("Stress Score", fontsize = 20)
ax3_s.set_title("As age increases, stress score tends to decrease",fontsize= 30)

In [91]:
#married_depression
grouped = df_classification.loc[:,['married', 'Depression_score']] \
    .groupby(['married']) \
    .median() \
    .sort_values(by='Depression_score')

ax4_d = sns.boxplot(x = df_classification["married"], y = df_classification["Depression_score"],order = grouped.index,data = df_classification)
ax4_d.set_xlabel("Marital Status", fontsize = 20)
ax4_d.set_ylabel("Depression Score", fontsize = 20)
ax4_d.set_title("Bachelor's tend to have higher Depression scores",fontsize= 30)
ax4_d.figure.savefig("married_d")

#married_anxiety

grouped = df_classification.loc[:,['married', 'Anxiety_score']] \
    .groupby(['married']) \
    .median() \
    .sort_values(by='Anxiety_score')
ax4_a = sns.boxplot(x = df_classification["married"], y = df_classification["Anxiety_score"],order = grouped.index,data = df_classification)
ax4_a.set_xlabel("Marital Status", fontsize = 20)
ax4_a.set_ylabel("Anxiety Score", fontsize = 20)
ax4_a.set_title("Bachelor's tend to have higher Anxiety scores",fontsize= 30)
ax4_a.figure.savefig("married_a")

#married_stress

grouped = df_classification.loc[:,['married', 'Stress_score']] \
    .groupby(['married']) \
    .median() \
    .sort_values(by='Stress_score')
ax4_s = sns.boxplot(x = df_classification["married"], y = df_classification["Stress_score"],order = grouped.index,data = df_classification)
ax4_s.set_xlabel("Marital Status", fontsize = 20)
ax4_s.set_ylabel("Stress Score", fontsize = 20)
ax4_s.set_title("Bachelor's tend to have higher Stress scores",fontsize= 30)
ax4_s.figure.savefig("married_s")

In [ ]:
ax6_d = sns.boxplot(x=df_classification['screensize'], y=df_classification['Depression_score'],data=df_classification)

ax6_d.set_xlabel("Screensize", fontsize = 20)
ax6_d.set_ylabel("Depression Score", fontsize = 20)
ax6_d.set_title("There is no difference in depression score for screensize",fontsize= 25)

In [ ]:
ax6_a = sns.boxplot(x=df_classification['screensize'], y=df_classification['Anxiety_score'],data=df_classification)
ax6_a.set_xlabel("Screensize", fontsize = 20)
ax6_a.set_ylabel("Anxiety Score", fontsize = 20)
ax6_a.set_title("Using mobile phones more, give higher anxiety",fontsize= 25)

In [ ]:
ax6_s = sns.boxplot(x=df_classification['screensize'], y=df_classification['Stress_score'],data=df_classification)
ax6_s.set_xlabel("Screensize", fontsize = 20)
ax6_s.set_ylabel("Stress Score", fontsize = 20)
ax6_s.set_title("Very small difference between mobile phones and laptops",fontsize= 25)

In [ ]:
ax7_d= sns.lineplot(x=df_classification['familysize'], y=df_classification['Depression_score'], ci=None,data=df_classification)
ax7_d.set_xlabel("Family Size", fontsize = 20)
ax7_d.set_ylabel("Depression Score", fontsize = 20)
ax7_d.set_title("Depreesion score increases with family size more than 20",fontsize= 25)

In [ ]:
ax7_a = sns.lineplot(x=df_classification['familysize'], y=df_classification['Anxiety_score'], ci=None,data=df_classification)
ax7_a.set_xlabel("Family Size", fontsize = 20)
ax7_a.set_ylabel("Anxiety Score", fontsize = 20)
ax7_a.set_title("Anxiety score increases with family size more than 16",fontsize= 25)

In [ ]:
ax7_s = sns.lineplot(x=df_classification['familysize'], y=df_classification['Stress_score'], ci=None,data=df_classification)
ax7_s.set_xlabel("Family Size", fontsize = 20)
ax7_s.set_ylabel("Stress Score", fontsize = 20)
ax7_s.set_title("Stress score increases with family size more than 20",fontsize= 25)

In [ ]:
#race_depression

grouped = df_classification.loc[:,['race', 'Depression_score']] \
    .groupby(['race']) \
    .median() \
    .sort_values(by='Depression_score')
ax8_d = sns.boxplot(x=df_classification['race'], y=df_classification['Depression_score'],order = grouped.index,data=df_classification)
ax8_d.set_xlabel("Race", fontsize = 20)
ax8_d.set_ylabel("Depression Score", fontsize = 20)
ax8_d.set_title("Indigenous Australians tend to have higher depression scores",fontsize= 25)
ax8_d.figure.savefig("race_d")

#race_anxiety


grouped = df_classification.loc[:,['race', 'Anxiety_score']] \
    .groupby(['race']) \
    .median() \
    .sort_values(by='Anxiety_score')
ax8_a = sns.boxplot(x=df_classification['race'], y=df_classification['Anxiety_score'],order = grouped.index,data=df_classification)
ax8_a.set_xlabel("Race", fontsize = 20)
ax8_a.set_ylabel("Anxiety Score", fontsize = 20)
ax8_a.set_title("Indigenous Australians tend to have higher Anxiety scores",fontsize= 25)
ax8_a.figure.savefig("race_a")

#race_stress

grouped = df_classification.loc[:,['race', 'Stress_score']] \
    .groupby(['race']) \
    .median() \
    .sort_values(by='Stress_score')
ax8_s = sns.boxplot(x=df_classification['race'], y=df_classification['Stress_score'],order=grouped.index,data=df_classification)
ax8_s.set_xlabel("Race", fontsize = 20)
ax8_s.set_ylabel("Stress Score", fontsize = 20)
ax8_s.set_title("Native Americans tend to have higher stress scores",fontsize= 25)
ax8_s.figure.savefig("race_s")

In [ ]:
#orientation_depression

grouped = df_classification.loc[:,['orientation', 'Depression_score']] \
    .groupby(['orientation']) \
    .median() \
    .sort_values(by='Depression_score')
ax9_d = sns.boxplot(x=df_classification['orientation'], y=df_classification['Depression_score'],order=grouped.index,data=df_classification)
ax9_d.set_xlabel("Sexual Orientation", fontsize = 20)
ax9_d.set_ylabel("Depression Score", fontsize = 20)
ax9_d.set_title("Bisexuals tend to have higher depression scores",fontsize= 25)
ax9_d.figure.savefig("orientation_d")

#orientation_anxiety

grouped = df_classification.loc[:,['orientation', 'Anxiety_score']] \
    .groupby(['orientation']) \
    .median() \
    .sort_values(by='Anxiety_score')
ax9_a= sns.boxplot(x=df_classification['orientation'], y=df_classification['Anxiety_score'],order=grouped.index,data=df_classification)
ax9_a.set_xlabel("Sexual Orientation", fontsize = 20)
ax9_a.set_ylabel("Anxiety Score", fontsize = 20)
ax9_a.set_title("Bisexuals tend to have little higher anxiety scores",fontsize= 25)
ax9_a.figure.savefig("orientation_a")

#orientation_stress

grouped = df_classification.loc[:,['orientation', 'Stress_score']] \
    .groupby(['orientation']) \
    .median() \
    .sort_values(by='Stress_score')
ax9_s = sns.boxplot(x=df_classification['orientation'], y=df_classification['Stress_score'],order= grouped.index,data=df_classification)
ax9_s.set_xlabel("Sexual Orientation", fontsize = 20)
ax9_s.set_ylabel("Stress Score", fontsize = 20)
ax9_s.set_title("Bisexuals tend to have higher stress scores",fontsize= 25)
ax9_s.figure.savefig("orientation_s")

In [ ]:

ax10_d = sns.boxplot(x=df_classification['urban'], y=df_classification['Depression_score'],data=df_classification)
ax10_d.set_xlabel("Type of Area", fontsize = 20)
ax10_d.set_ylabel("Depression Score", fontsize = 20)
ax10_d.set_title("No difference between types of areas in depression scores",fontsize= 25)


In [ ]:

ax10_a = sns.boxplot(x=df_classification['urban'], y=df_classification['Anxiety_score'],data=df_classification)
ax10_a.set_xlabel("Type of Area", fontsize = 20)
ax10_a.set_ylabel("Depression Score", fontsize = 20)
ax10_a.set_title("Urban areas have a little higher anxiety scores",fontsize= 25)


In [ ]:

ax10_s = sns.boxplot(x=df_classification['urban'], y=df_classification['Stress_score'],data=df_classification)
ax10_s.set_xlabel("Type of Area", fontsize = 20)
ax10_s.set_ylabel("Depression Score", fontsize = 20)
ax10_s.set_title("No difference between types of areas in stress scores",fontsize= 25)

In [ ]:
df_classification['education']

In [ ]:

df_classification["education"].replace({0: 3}, inplace=True)
df_classification["education"].replace({1: 'Below High School', 2: 'High School', 3 : 'Undergraduate',4:'Graduate'}, inplace=True)
print(df_classification['education'].value_counts())

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style("whitegrid")
grouped = df_classification.loc[:,['education', 'Depression_score']] \
    .groupby(['education']) \
    .median() \
    .sort_values(by='Depression_score')
ax11_d = sns.boxplot(x=df_classification['education'], y=df_classification['Depression_score'],order= grouped.index,data=df_classification)
ax11_d.set_xlabel("Education Level", fontsize = 20)
ax11_d.set_ylabel("Depression Score", fontsize = 20)
ax11_d.set_title("People having education below high school have high depression",fontsize= 25)
ax11_d.figure.savefig("education_d")

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style("whitegrid")
grouped = df_classification.loc[:,['education', 'Anxiety_score']] \
    .groupby(['education']) \
    .median() \
    .sort_values(by='Anxiety_score')
ax11_a = sns.boxplot(x=df_classification['education'], y=df_classification['Anxiety_score'],order= grouped.index,data=df_classification)
ax11_a.set_xlabel("Education Level", fontsize = 20)
ax11_a.set_ylabel("Anxiety Score", fontsize = 20)
ax11_a.set_title("People having education below high school have high Anxiety",fontsize= 25)
ax11_a.figure.savefig("education_a")

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style("whitegrid")
grouped = df_classification.loc[:,['education', 'Stress_score']] \
    .groupby(['education']) \
    .median() \
    .sort_values(by='Stress_score')
ax11_s = sns.boxplot(x=df_classification['education'], y=df_classification['Stress_score'],order= grouped.index,data=df_classification)
ax11_s.set_xlabel("Education Level", fontsize = 20)
ax11_s.set_ylabel("Stress Score", fontsize = 20)
ax11_s.set_title("People having education below high school have high Stress",fontsize= 25)
ax11_s.figure.savefig("education_s")

Predicting Depression using Machine learning

In [ ]:
from sklearn.model_selection import train_test_split


y = np.array((df_regression['Depression_score']))
print(y)
feature_corr = df_regression
train = df_regression.drop(['Depression_score'], axis=1)

from sklearn.preprocessing import StandardScaler
num_features = train.select_dtypes(exclude = ["object"]).columns
stdSc = StandardScaler()
train.loc[:, num_features] = stdSc.fit_transform(train.loc[:, num_features])
y = y.reshape(-1,1)
print(y)
y = stdSc.fit_transform(y)
#y = train['Depression_score']
#train = df_regression.drop(['Depression_score'], axis=1)

#print(train)

X_train, X_hold_out, y_train, y_hold_out = train_test_split(train, y, test_size = 0.3, random_state = 42)
y_hold_out_final = y_hold_out

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42)
#print(X_train)


In [ ]:
feature_corr.dtypes



Feature Extraction

In [ ]:
print("Find most important features relative to target")
corr = feature_corr.corr()
corr.sort_values(["Depression_score"], ascending = False, inplace = True)
#print(corr.SalePrice)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(corr.Depression_score)

In [ ]:
#from sklearn.preprocessing import StandardScaler
#num_features = train.select_dtypes(exclude = ["object"]).columns
#stdSc = StandardScaler()
#X_train.loc[:, num_features] = stdSc.fit_transform(X_train.loc[:, num_features])
#X_test.loc[:, num_features] = stdSc.fit_transform(X_test.loc[:, num_features])
#X_hold_out.loc[:,num_features] = stdSc.fit_transform(X_hold_out.loc[:,num_features])
#y_train = stdSc.fit_transform(y_train)
#y_test = stdSc.fit_transform(y_test)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score


ridge = Ridge()

lasso = Lasso()

elasticnet = ElasticNet()

gbr = GradientBoostingRegressor()

lightgbm = LGBMRegressor()

xgboost = XGBRegressor()

scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                           meta_regressor=xgboost,
                           use_features_in_secondary=True)

In [ ]:
"""from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
param_grid = [{'alpha': [0.0001,0.001,0.01,0.01,1,10]}]
grid_search = GridSearchCV(sgd, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)
"""



In [ ]:
"""from sklearn.model_selection import GridSearchCV
param_grid = [{'alpha': [0.0001,0.001,0.01,0.01,1,10]}]
grid_search = GridSearchCV(lasso, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)"""

In [ ]:
"""param_grid = [{'alpha': [0.01,1,10,100,1000]}]
grid_search = GridSearchCV(ridge, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)"""

In [ ]:
"""param_grid = [{'alpha': [0.0001,0.001,0.01,0.01,1,10]}]
grid_search = GridSearchCV(elasticnet, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)"""

In [ ]:
"""param_grid = [{'n_estimators' : [100], 'learning_rate':[0.05,0.1], 'max_depth':[4,8], 'max_features':['sqrt',8],
              'min_samples_leaf':[15], 'min_samples_split':[10], 'loss':['huber']}]
grid_search = GridSearchCV(gbr, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)"""

In [ ]:
"""param_grid = [{'num_leaves':[4,40,100], 'learning_rate': [0.01,0.1,1], 'n_estimators':[5000,500]}]
grid_search = GridSearchCV(lightgbm, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)"""

In [ ]:
"""param_grid = [{'learning_rate' : [0.01,1] ,'n_estimators' : [1000,3500], 'max_depth' : [3,5]}]
grid_search = GridSearchCV(xgboost, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)"""

In [ ]:
lasso = Lasso(alpha = 0.001)
ridge = Ridge(alpha = 10)
elasticnet = ElasticNet(alpha = 0.001)
gbr = GradientBoostingRegressor(learning_rate = 0.05, loss = 'huber', max_depth = 8, max_features =8,
                                min_samples_leaf = 15, min_samples_split = 10, n_estimators =100)
lightgbm = LGBMRegressor(learning_rate = 0.1, n_estimators = 500, num_leaves = 4)
xgboost = XGBRegressor(learning_rate = 0.01, max_depth = 3, n_estimators = 3500)

In [ ]:
scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)
rmse_list = []
models = [lasso,ridge,elasticnet,gbr,lightgbm,xgboost]

scvr.fit(np.array(X_train),np.array(y_train))
y_pred = scvr.predict(np.array(X_test))
rmse_list.append(mean_squared_error(y_test,y_pred,squared = False))
for i in models:
    i.fit(X_train,y_train)
    y_pred = i.predict(X_test)
    rmse_list.append(mean_squared_error(y_test,y_pred,squared = False))
print(rmse_list)

#kfold = KFold(n_splits=5, random_state=7, shuffle=True)
#score = cross_val_score(lasso, X_hold_out, y_hold_out, cv= kfold,scoring='neg_mean_squared_error')

#scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
 #                               meta_regressor=xgboost,
  #                              use_features_in_secondary=True)
#print(np.sqrt(-score))
#print(lasso.predict(X_hold_out))

In [ ]:
def blend_models_predict1(X):
    return (scvr.predict(np.array(X)))
            #+ \
            #(0.1 * lasso.predict(X)) + \
            #(0.1 * ridge.predict(X)) + \
            #(0.2 * gbr.predict(X)) + \
            #(0.2 * xgboost.predict(X)) + \
            #(0.1 * lightgbm.predict(X)) + \
            #(0.2 * scvr.predict(np.array(X)))

In [ ]:
y_pred = blend_models_predict1(X_hold_out)
y_pred


In [ ]:
y_pred = pd.DataFrame(y_pred, columns = ['target'])
print(y_pred)
y_pred_final = stdSc.inverse_transform(y_pred)
print(y_pred_final)
y_hold_out_final = y_hold_out
y_hold_out_final = stdSc.inverse_transform(y_hold_out_final)
print(y_hold_out_final)


In [ ]:
#confusion_matrix(y_holdout, y_pred, labels=range(n_classes)
np.sqrt(mean_squared_error(y_hold_out_final, y_pred_final))


Predicting Anxiety


In [ ]:
from sklearn.model_selection import train_test_split


y = np.array((df_regression['Anxiety_score']))
print(y)
feature_corr = df_regression
train = df_regression.drop(['Anxiety_score'], axis=1)

from sklearn.preprocessing import StandardScaler
num_features = train.select_dtypes(exclude = ["object"]).columns
stdSc = StandardScaler()
train.loc[:, num_features] = stdSc.fit_transform(train.loc[:, num_features])
y = y.reshape(-1,1)
print(y)
y = stdSc.fit_transform(y)
#y = train['Depression_score']
#train = df_regression.drop(['Depression_score'], axis=1)

#print(train)

X_train, X_hold_out, y_train, y_hold_out = train_test_split(train, y, test_size = 0.3, random_state = 42)
y_hold_out_final = y_hold_out

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42)
#print(X_train)


In [ ]:
feature_corr.dtypes

In [ ]:
print("Find most important features relative to target")
corr = feature_corr.corr()
corr.sort_values(["Anxiety_score"], ascending = False, inplace = True)
#print(corr.SalePrice)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(corr.Anxiety_score)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score


ridge = Ridge()

lasso = Lasso()

elasticnet = ElasticNet()

gbr = GradientBoostingRegressor()

lightgbm = LGBMRegressor()

xgboost = XGBRegressor()

scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                           meta_regressor=xgboost,
                           use_features_in_secondary=True)

In [ ]:
'''from sklearn.model_selection import GridSearchCV
param_grid = [{'alpha': [0.0001,0.001,0.01,0.01,1,10]}]
grid_search = GridSearchCV(lasso, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
'''param_grid = [{'alpha': [0.01,1,10,100,1000]}]
grid_search = GridSearchCV(ridge, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
'''param_grid = [{'alpha': [0.0001,0.001,0.01,0.01,1,10]}]
grid_search = GridSearchCV(elasticnet, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
'''param_grid = [{'n_estimators' : [100], 'learning_rate':[0.05,0.1], 'max_depth':[4,8], 'max_features':['sqrt',8],
              'min_samples_leaf':[15], 'min_samples_split':[10], 'loss':['huber']}]
grid_search = GridSearchCV(gbr, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
'''param_grid = [{'num_leaves':[4,40,100], 'learning_rate': [0.01,0.1,1], 'n_estimators':[5000,500]}]
grid_search = GridSearchCV(lightgbm, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
'''param_grid = [{'learning_rate' : [0.01,1] ,'n_estimators' : [1000,3500], 'max_depth' : [3,5]}]
grid_search = GridSearchCV(xgboost, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
lasso = Lasso(alpha = 0.001)
ridge = Ridge(alpha = 10)
elasticnet = ElasticNet(alpha = 0.001)
gbr = GradientBoostingRegressor(learning_rate = 0.05, loss = 'huber', max_depth = 8, max_features =8,
                                min_samples_leaf = 15, min_samples_split = 10, n_estimators =100)
lightgbm = LGBMRegressor(learning_rate = 0.1, n_estimators = 500, num_leaves = 4)
xgboost = XGBRegressor(learning_rate = 0.01, max_depth = 3, n_estimators = 3500)

In [ ]:
scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)
rmse_list = []
models = [lasso,ridge,elasticnet,gbr,lightgbm,xgboost]

scvr.fit(np.array(X_train),np.array(y_train))
y_pred = scvr.predict(np.array(X_test))
rmse_list.append(mean_squared_error(y_test,y_pred,squared = False))
for i in models:
    i.fit(X_train,y_train)
    y_pred = i.predict(X_test)
    rmse_list.append(mean_squared_error(y_test,y_pred,squared = False))
print(rmse_list)

#kfold = KFold(n_splits=5, random_state=7, shuffle=True)
#score = cross_val_score(lasso, X_hold_out, y_hold_out, cv= kfold,scoring='neg_mean_squared_error')

#scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
 #                               meta_regressor=xgboost,
  #                              use_features_in_secondary=True)
#print(np.sqrt(-score))
#print(lasso.predict(X_hold_out))

In [ ]:
def blend_models_predict1(X):
    return (scvr.predict(np.array(X)))
            #+ \
            #(0.1 * lasso.predict(X)) + \
            #(0.1 * ridge.predict(X)) + \
            #(0.2 * gbr.predict(X)) + \
            #(0.2 * xgboost.predict(X)) + \
            #(0.1 * lightgbm.predict(X)) + \
            #(0.2 * scvr.predict(np.array(X)))

In [ ]:
y_pred = blend_models_predict1(X_hold_out)
y_pred


In [ ]:
y_pred = pd.DataFrame(y_pred, columns = ['target'])
print(y_pred)
y_pred_final = stdSc.inverse_transform(y_pred)
print(y_pred_final)
y_hold_out_final = y_hold_out
y_hold_out_final = stdSc.inverse_transform(y_hold_out_final)
print(y_hold_out_final)

In [ ]:
#confusion_matrix(y_holdout, y_pred, labels=range(n_classes)
np.sqrt(mean_squared_error(y_hold_out_final, y_pred_final))

Predicting Stress


In [ ]:
from sklearn.model_selection import train_test_split


y = np.array((df_regression['Stress_score']))
print(y)
feature_corr = df_regression
train = df_regression.drop(['Stress_score'], axis=1)

from sklearn.preprocessing import StandardScaler
num_features = train.select_dtypes(exclude = ["object"]).columns
stdSc = StandardScaler()
train.loc[:, num_features] = stdSc.fit_transform(train.loc[:, num_features])
y = y.reshape(-1,1)
print(y)
y = stdSc.fit_transform(y)
#y = train['Depression_score']
#train = df_regression.drop(['Depression_score'], axis=1)

#print(train)

X_train, X_hold_out, y_train, y_hold_out = train_test_split(train, y, test_size = 0.3, random_state = 42)
y_hold_out_final = y_hold_out

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42)
#print(X_train)


In [ ]:
feature_corr.dtypes

In [ ]:
print("Find most important features relative to target")
corr = feature_corr.corr()
corr.sort_values(["Stress_score"], ascending = False, inplace = True)
#print(corr.SalePrice)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(corr.Stress_score)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score

ridge = Ridge()

lasso = Lasso()

elasticnet = ElasticNet()

gbr = GradientBoostingRegressor()

lightgbm = LGBMRegressor()

xgboost = XGBRegressor()

scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                           meta_regressor=xgboost,
                           use_features_in_secondary=True)

In [ ]:
'''from sklearn.model_selection import GridSearchCV
param_grid = [{'alpha': [0.0001,0.001,0.01,0.01,1,10]}]
grid_search = GridSearchCV(lasso, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:
'''param_grid = [{'alpha': [0.01,1,10,100,1000]}]
grid_search = GridSearchCV(ridge, param_grid, cv=5,scoring='neg_mean_squared_error',return_train_score=True)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
grid_search.score(X_test,y_test)'''

In [ ]:

lasso = Lasso(alpha = 0.001)
ridge = Ridge(alpha = 10)
elasticnet = ElasticNet(alpha = 0.001)
gbr = GradientBoostingRegressor(learning_rate = 0.05, loss = 'huber', max_depth = 8, max_features =8,
                                min_samples_leaf = 15, min_samples_split = 10, n_estimators =100)
lightgbm = LGBMRegressor(learning_rate = 0.1, n_estimators = 500, num_leaves = 4)
xgboost = XGBRegressor(learning_rate = 0.01, max_depth = 3, n_estimators = 3500)


In [ ]:
scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)
rmse_list = []
models = [lasso,ridge,elasticnet,gbr,lightgbm,xgboost]

scvr.fit(np.array(X_train),np.array(y_train))
y_pred = scvr.predict(np.array(X_test))
rmse_list.append(mean_squared_error(y_test,y_pred,squared = False))
for i in models:
    i.fit(X_train,y_train)
    y_pred = i.predict(X_test)
    rmse_list.append(mean_squared_error(y_test,y_pred,squared = False))
print(rmse_list)

#kfold = KFold(n_splits=5, random_state=7, shuffle=True)
#score = cross_val_score(lasso, X_hold_out, y_hold_out, cv= kfold,scoring='neg_mean_squared_error')

#scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
 #                               meta_regressor=xgboost,
  #                              use_features_in_secondary=True)
#print(np.sqrt(-score))
#print(lasso.predict(X_hold_out))

In [ ]:
def blend_models_predict1(X):
    return (scvr.predict(np.array(X)))
            #+ \
            #(0.1 * lasso.predict(X)) + \
            #(0.1 * ridge.predict(X)) + \
            #(0.2 * gbr.predict(X)) + \
            #(0.2 * xgboost.predict(X)) + \
            #(0.1 * lightgbm.predict(X)) + \
            #(0.2 * scvr.predict(np.array(X)))

In [ ]:
y_pred = blend_models_predict1(X_hold_out)
y_pred


In [ ]:
y_pred = pd.DataFrame(y_pred, columns = ['target'])
print(y_pred)
y_pred_final = stdSc.inverse_transform(y_pred)
print(y_pred_final)
y_hold_out_final = y_hold_out
y_hold_out_final = stdSc.inverse_transform(y_hold_out_final)
print(y_hold_out_final)

In [ ]:
#confusion_matrix(y_holdout, y_pred, labels=range(n_classes)
np.sqrt(mean_squared_error(y_hold_out_final, y_pred_final))